In [4]:
from IPython.display import HTML, display

In [5]:
%load_ext Cython

In [6]:
%%cython
# cython: infer_types=True, annotation_typing=True
## cython: infer_types.verbose=True 
from IPython.display import HTML, display
cdef class Connect4:
    cdef public:
        long turn
        long long[2] data
    cpdef long get_col_row(self, col: long, row: long):
        cdef long long mask
        pos = col * 7 + row
        mask = (<long long>1) << pos 
        if self.data[1] & mask:
            return 2
        return bool(self.data[0] & mask)
    
    cpdef long is_end(self):
        cdef long long bitboard, bound, mask
        bitboard = self.data[1-self.turn%2]
        bound = (<long long>1)<<48 # 49 = 7*(6+1)  
        # horizontal: 0x204081 = 1|(1<<7)|(1<<14)|(1<<21)
        # vertical: 0xf = 1|(1<<1)|(1<<2)|(1<<3)
        # up-right: 0x1010101 = 1|(1<<8)|(1<<16)|(1<<24)
        # down-right: 0x208208 = (1<<3)|(1<<9)|(1<<15)|(1<<21)
        for mask in [0x204081, 0xf,  0x1010101, 0x208208]:
            while mask < bound:
                if mask & bitboard == mask:
                    return True
                mask <<= 1
        return False
    
    cpdef set_col_row(self, col:long, row:long, value:long):
        # assert value in [0,1,2]
        pos = col * 7 + row
        mask = (<long long>1) << pos
        neg_mask = ~mask       
        if value == 1 or value ==2:
            self.data[value-1] |= mask
            self.data[2-value] &= neg_mask
        else:
            self.data[0] &= neg_mask
            self.data[1] &= neg_mask
            
    def __init__(self, data=None, turn=0):
        if data is not None:
            self.data = data[:]
        else:
            self.data = [0, 0]
        self.turn = turn
        
    cpdef remove(self, col:long):
        shift = col*7
        mask = (((self.data[0]|self.data[1]) >> shift) &0x3f) +1
        mask = (mask >> 1) << shift
        # print(shift, hex(mask), hex(self.data[0]), hex(self.data[1]))
        neg_mask = ~mask
        self.data[0] &= neg_mask
        self.data[1] &= neg_mask
        
    cpdef move(self, col:long, test=False):
        # assert 0<= col <7
        shift = col*7
        mask = (((self.data[0]|self.data[1]) >> shift) &0x3f) +1
        # print("mask=", mask)
        if mask >= 64:
            return None
        if not test:
            self.data[self.turn%2] |= (mask<<shift)
            self.turn += 1
        return self
    
    def board_data(self):
        for i in range(7):
            for j in range(6):
                c = self.get_col_row(i,j)
                if c!=0:
                    yield i,j,c
                    
    def _repr_html_(self):
        def pos(i):
            return int(7+(220-6.5)*i/8)
        imgstr = "<img src='img/%s.png' width='23px' height='23px' style='position: absolute; top: %spx; left: %spx;margin-top: 0;z-index: %d' />"
        header = """<div style="width: 200px; height:180px;position: relative;background: blue">"""
        header += "\n".join(imgstr%('empty', pos(5-j), pos(i), 0) for i in range(7) for j in range(6))
        return header +"\n".join(imgstr%('red_coin' if c==1 else 'yellow_coin', pos(5-j), pos(i), 2) for (i,j,c) in self.board_data()) +"</div>"
    
    def display(self):
        display(HTML(self._repr_html_()))
    
    def __repr__(self):
        row_str = lambda j: "".join(".ox"[self.get_col_row(i,j)] for i in range(7))
        return "\n".join(row_str(j) for j in range(5,-1,-1))

from random import randint
def random_play(init_data=None, init_turn=0, display=False):
    game = Connect4(init_data, init_turn)
    while game.turn < 42 and not game.is_end():
        while game.move(randint(0,6)) is None:
            continue
    if display:
        game.display()
    if game.is_end():
        return game.turn
    return 0

/usr/local/lib/python3.4/dist-packages/IPython/utils/path.py:264: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [7]:
def MC_agent(_game, N=500):
    score = [-1.0*N]*7
    for i in range(7):
        game = Connect4(_game.data, _game.turn)
        if game.move(i):
            if game.is_end():
                return i
            s = 0
            for j in range(N):
                #print("move", i, "case", j)
                r = random_play(game.data, game.turn)
                turn = (r-1)%2
                if r == 0:
                    pass
                elif  (r-1)%2 == _game.turn%2:
                    s += 0.95** (r-_game.turn-1)
                else:
                    s -= .95** (r-_game.turn-1)
            score[i] = s/N
    return max(zip(score, range(7)))[1]

In [8]:
def random_vs_MC(init_data=None, init_turn=0, display=False):
    game = Connect4(init_data, init_turn)
    while game.turn < 42 and not game.is_end():
        if game.turn%2 == 0:
            while game.move(randint(0,6)) is None:
                continue
        else:
            i = MC_agent(game)
            game.move(i)
        if display == 'all':
            game.display()
    if display:
        game.display()
    if game.is_end():
        return game.turn
    return 0

In [9]:
from random import choice, random, shuffle
Q=[None]*2**18
reg = lambda x: 1. if x is None else -x

In [ ]:
def run_game(Q, V, verbose = False, epsilon=0.1, lmbd=0.95):
    game = Connect4()
    history=[]
    while game.turn < 42 and not game.is_end():
        s = 1 if game.turn%2 == 0 else -1
        moves = [i for i in range(7) if game.move(i, test=True)]
        if random() < epsilon:
            m = choice(moves)
        else:
            values = [s*Q(game, i) for i in moves]
            m = max(zip(values,moves))[1]
        game.move(m)
        history.append(tuple(game.data))
    if game.is_end():
        result = 1 if (game.turn-1)%2 == 0 else -1
    else:
        result = 0 # Tie
    # train here
    estimate_V = [0.]*len(history)
    r = result*1.0
    for i in range(len(history)-1, -1):
        estimate_V[i]=r
        r *= lmbd
        
for i in range(100):
    run_game(Game())
run_game(Game(), True)

In [10]:
import numpy as np
import theano
import theano.tensor as T

import lasagne

In [ ]:
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')
l_in = lasagne.layers.InputLayer(shape=(None, 2, 6, 7), input_var=input_var)

l_out = lasagne.layers.DenseLayer(l_in, num_units=1, nonlinearity=lasagne.nonlinearities.tanh, W=lasagne.init.GlorotUniform())

target_var = T.scalar()

prediction = lasagne.layers.get_output(l_out)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [11]:
l_out

NameError: name 'l_out' is not defined